# Changes from v11
* Use UMLS to train


# Import the MIMIC data

In [44]:
import gzip
import pandas as pd
import glob

dataset_dictionary = {}

for file_path in glob.glob('.\\Data\\MIMIC Files\*'):
    file_name = file_path.split('\\')[3].split('.')[0]
    with gzip.open(file_path, mode='r') as file:
        dataset_dictionary[file_name] = pd.read_csv(file)

c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Join the tables

In [45]:
# Create dataset to join together -----

# Create note_events table -----

# Combine text for each subject and encounter
note_events_base = dataset_dictionary['NOTEEVENTS'][dataset_dictionary['NOTEEVENTS'].loc[:,'CATEGORY'] == 'Discharge summary']
note_events = note_events_base.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False)['TEXT'].agg(sum)

# Create CPT table -----

cpt_events_base = dataset_dictionary['CPTEVENTS']
cpt_events_base = cpt_events_base[cpt_events_base['TICKET_ID_SEQ'] == 1]
cpt_events_base = cpt_events_base.loc[:, ['SUBJECT_ID','HADM_ID', 'CPT_CD']]
cpt_events = cpt_events_base.drop_duplicates()
cpt_events

# Join the datasets -----

note_cpt = note_events.merge(cpt_events, on = ['SUBJECT_ID','HADM_ID'])
# print(note_cpt.shape, note_events.shape, cpt_events.shape) # (223,150, 4) (52,726, 3) (227,510, 3)

In [50]:
# Export 25 notes for 4 CPT descriptions

# note_cpt[note_cpt['CPT_CD'].astype(str) == '33518'].head(10).loc[:, ['CPT_CD','TEXT']]

# Sample_Metamap = {}

for i in ['33518', '36556', '99221', '99291']:
#     csv_nm = i + '_text.csv'
    df_nm = i + '_df'
    Sample_Metamap[df_nm] = pd.DataFrame(note_cpt[note_cpt['CPT_CD'].astype(str) == i].head(10).loc[:, ['CPT_CD','TEXT']])
#     note_cpt[note_cpt['CPT_CD'].astype(str) == '33518'].head(10).loc[:, ['CPT_CD','TEXT']].to_csv(csv_nm)

In [51]:
Sample_Metamap['33518_df']

,CPT_CD,TEXT
421,33518,Admission Date: [**2151-8-25**] ...
652,33518,Admission Date: [**2184-1-15**] ...
1440,33518,Admission Date: [**2141-6-30**] ...
1487,33518,Admission Date: [**2161-2-12**] ...
1962,33518,Admission Date: [**2189-3-27**] ...
2339,33518,Admission Date: [**2190-2-18**] ...
3577,33518,Admission Date: [**2162-4-22**] ...
4212,33518,Admission Date: [**2144-10-14**] ...
4604,33518,Admission Date: [**2172-12-14**] ...
4848,33518,Admission Date: [**2171-2-21**] Dischar...


# Filter the data to CPT over 200 samples + Resample

In [92]:
import numpy as np
from sklearn.utils import resample

# Value Counts
print(note_cpt['CPT_CD'].astype(str).value_counts())

# Filter to CPT with over 200 notes
df = note_cpt['CPT_CD'].astype(str).value_counts()
top_200 = list((df[df > 200]).index.values)
note_cpt_4 = note_cpt[note_cpt['CPT_CD'].astype(str).isin(top_200)]

# Resample minority groups -----

# Remove largest group
top_200.remove('99291')
'99291' in top_200

minority_ls = top_200
# minority_ls = ['99223','99222','99254']

minority_df = []
for i in minority_ls:
    test_resampled = resample(note_cpt[note_cpt['CPT_CD'].astype(str) == i], replace=True, n_samples=7860, random_state=123)
    minority_df.append(test_resampled)

minority_df.append(note_cpt[note_cpt['CPT_CD'].astype(str) == '99291'])
new_df = pd.concat(minority_df)

new_df['CPT_CD'] = new_df['CPT_CD'].astype(str)

99291    7860
99223    2851
99222    1736
99254    1242
99255     882
         ... 
33542       1
22630       1
26418       1
63103       1
56620       1
Name: CPT_CD, Length: 707, dtype: int64


# Check for Imbalance

In [93]:
# import matplotlib.pyplot as plt

# plt.hist(note_cpt['CPT_CD'].astype(str))
# plt.show()

new_df['CPT_CD'].value_counts()

99252    7860
33508    7860
99222    7860
99291    7860
99253    7860
99233    7860
99254    7860
99231    7860
33405    7860
99292    7860
33533    7860
99255    7860
36556    7860
99221    7860
99232    7860
99223    7860
Name: CPT_CD, dtype: int64

# Filter the data

In [94]:
import string

def clean_data(text_series):
    
    # Replace \n 
    text_series = text_series.str.replace('\\n',' ', regex=True)    

    # Remove dates and locations
    text_series = text_series.str.replace('\[\*\*(.*?)\*\*\]', ' ', regex=True)
    
    # Remove topics
    data = text_series.str.split('([A-Z\s]+:)')
    for row_num, value in enumerate(data):
        text_chunks = [x.strip().replace(':','').replace('\n', '') for x in value]
        for i, x in enumerate(text_chunks):
            if 'MEDICATION' in x or 'SOCIAL HISTORY' in x or 'FAMILY HISTORY' in x:
                text_chunks[i] = ' '
                try:
                    text_chunks[i + 1] = ' '
                except:
                    continue

        text_series.iloc[row_num] = ' '.join(text_chunks)
    
    # Replace punctuation
    text_series = text_series.str.replace('[' + string.punctuation + ']', ' ', regex=True)
    
    # Convert to lowercase 
    text_series = text_series.str.lower()
    
    # Remove all digits
    text_series = text_series.str.replace('\d',' ', regex=True)
    
    # Replace plurals, endings with ing, endings with ed, endings with ly
#     text_series = text_series.str.replace('s(?=\s)', ' ', regex=True)
#     text_series = text_series.str.replace('ing(?=\s)', ' ', regex=True)
#     text_series = text_series.str.replace('ed(?=\s)', ' ', regex=True)
#     text_series = text_series.str.replace('ly(?=\s)', ' ', regex=True)
    
    return text_series

# Update Text Column

new_df.loc[:, 'TEXT'] = clean_data(new_df['TEXT']).values

# for i in Sample_Metamap.keys():
#     Sample_Metamap[i]['TEXT'] = clean_data(Sample_Metamap[i]['TEXT'])

In [59]:
for i in Sample_Metamap.keys():
    Sample_Metamap[i].to_csv(i + '.csv')

# Import and Concatenate the Datasets

In [85]:
Sample_Metamap.keys()
df_33518 = pd.read_csv('33518_df.csv') 
df_36556 = pd.read_csv('36556_df.csv') 
df_99221 = pd.read_csv('99221_df.csv') 
df_99291 = pd.read_csv('99291_df.csv') 


In [70]:
for i in [df_99291, df_99221, df_36556, df_33518]:
    i.drop(0, inplace=True)
    

In [86]:
main_df = pd.concat([df_99291, df_99221, df_36556, df_33518], axis=0)

In [88]:
main_df.drop('Unnamed: 0', inplace=True, axis=1)

# Shuffle the Data

In [7]:
new_df = new_df.sample(n = len(new_df), random_state = 42)

In [92]:
main_df = main_df.sample(n = len(main_df), random_state = 42)

In [95]:
main_df['Metamap_TEXT']

,CPT_CD,TEXT,Metamap_TEXT
9,99221,admission date discharge date date ...,"C0022877: laboratory: Laboratory : [mnob, hcro..."
6,99221,admission date discharge date date ...,C0020889: iliac: Bone structure of ilium : [bp...
5,99221,unit no admission date discharge date d...,"C0687676: after: Post : [tmco]: [LNC, MTH, CHV..."
6,36556,admission date discharge date ...,"C2348382: effect: Effect, Appearance : [qlco]:..."
4,99291,admission date discharge date ...,C1880497: free: Empty (qualifier) : [qlco]: [M...
2,99221,service date da...,C0032790: postoperative: Postoperative Period ...
7,33518,admission date discharge date ...,"C0018787: cardiac: Heart : [bpoc]: [LNC, MTH, ..."
7,36556,admission date discharge date ...,C0524850: neurosurgery: Neurosurgical Procedur...
9,33518,admission date discharge date date...,"C0205266: intact: Intact : [qlco]: [LNC, MTH, ..."
6,99291,admission date discharge date ...,C3526598: psychiatric: Psychiatric service : [...


# Clean Metamap Text

In [98]:
main_df['Metamap_TEXT'].str.split(expand=True)

,0,1,2,3,4,5,6,7,8,9,...,5757,5758,5759,5760,5761,5762,5763,5764,5765,5766
9,C0022877:,laboratory:,Laboratory,:,"[mnob,",hcro]:,"[LNC,","MTH,","CHV,","LCH,",...,None,None,None,None,None,None,None,None,None,None
6,C0020889:,iliac:,Bone,structure,of,ilium,:,[bpoc]:,"[MTH,","LCH,",...,None,None,None,None,None,None,None,None,None,None
5,C0687676:,after:,Post,:,[tmco]:,"[LNC,","MTH,","CHV,","NCI_CDISC,",NCI]:,...,None,None,None,None,None,None,None,None,None,None
6,C2348382:,effect:,"Effect,",Appearance,:,[qlco]:,"[MTH,",NCI]:,1977,:,...,None,None,None,None,None,None,None,None,None,None
4,C1880497:,free:,Empty,(qualifier),:,[qlco]:,"[MTH,","LNC,",NCI]:,3014,...,None,None,None,None,None,None,None,None,None,None
2,C0032790:,postoperative:,Postoperative,Period,:,[tmco]:,"[MTH,","CHV,","LNC,","MSH,",...,None,None,None,None,None,None,None,None,None,None
7,C0018787:,cardiac:,Heart,:,[bpoc]:,"[LNC,","MTH,","CSP,","MSH,","OMIM,",...,None,None,None,None,None,None,None,None,None,None
7,C0524850:,neurosurgery:,Neurosurgical,Procedures,:,[topp]:,"[MTH,","CHV,","LNC,","CCS_10,",...,None,None,None,None,None,None,None,None,None,None
9,C0205266:,intact:,Intact,:,[qlco]:,"[LNC,","MTH,","CHV,","NCI_FDA,",NCI]:,...,None,None,None,None,None,None,None,None,None,None
6,C3526598:,psychiatric:,Psychiatric,service,:,[topp]:,"[MTH,",HL7V3.0]:,1673,:,...,None,None,None,None,None,None,None,None,None,None


In [320]:
test_df = main_df.copy(deep=True).reset_index()

In [321]:
test_df['Metamap_TEXT'] = test_df['Metamap_TEXT'].str.split('\n').reset_index(drop=True)

blank_ls = []
cpt_cd_ref = []

# 40 documents
for index, doc in enumerate(test_df['Metamap_TEXT']):
#     for each line in the list
        blank_ls.append([line.split(':') for line in doc])
        cpt_cd_ref.append(test_df['CPT_CD'][index])
        
#     
# line.split(':')

In [322]:
# pd.concat([mz,nz], axis=0)
# pd.DataFrame(blank_ls[0][0]).transpose()
cpt_cd_ref

[99221,
 99221,
 99221,
 36556,
 99291,
 99221,
 33518,
 36556,
 33518,
 99291,
 36556,
 99291,
 99221,
 33518,
 33518,
 99291,
 99221,
 36556,
 99291,
 33518,
 99291,
 99221,
 99291,
 36556,
 36556,
 99291,
 33518,
 33518,
 99291,
 33518,
 36556,
 33518,
 99221,
 36556,
 99221,
 36556,
 99291,
 99221,
 36556,
 33518]

In [323]:
metamap_ls = []

for index, doc in enumerate(blank_ls):
    cpt_cd = cpt_cd_ref[index]
    for line in doc:
        x = pd.DataFrame(line).transpose()
        x['CPT_CD'] = cpt_cd
        x['INDEX'] = index
        metamap_ls.append(x)


In [324]:
metamap_df = pd.concat(metamap_ls)

In [325]:
np.unique(metamap_df['CPT_CD'].astype(str))

array(['33518', '36556', '99221', '99291'], dtype=object)

In [326]:
# df.groupby(['INDEX'])['text'].apply(lambda x: ','.join(x)).reset_index()
y = metamap_df[metamap_df[3].astype(str) == ' [topp]']
# import numpy as np

np.unique(metamap_df['CPT_CD'].astype(str))

array(['33518', '36556', '99221', '99291'], dtype=object)

In [329]:
z = y.groupby(['INDEX','CPT_CD'])[2].apply(lambda x: ','.join(x)).reset_index()

In [330]:
z

,INDEX,CPT_CD,2
0,0,99221,"stent placement , Facilitated oscillatory rel..."
1,1,99221,"Excision , Facilitated oscillatory release te..."
2,2,99221,"Facilitated oscillatory release technique , C..."
3,3,36556,"Invasive procedure , placement of tube , Chan..."
4,4,99291,"Fertilization in Vitro , Treatment Protocols ..."
5,5,99221,"Therapeutic procedure , Physical therapy , Fa..."
6,6,33518,"Operative Surgical Procedures , Facilitated o..."
7,7,36556,"Neurosurgical Procedures , Repositioning (pro..."
8,8,33518,"Facilitated oscillatory release technique , H..."
9,9,99291,"Psychiatric service , Facilitated oscillatory..."


# Clean Z

In [332]:
z[2] = clean_data(z[2])

In [336]:
# z.iloc[0,2]

'stent placement   facilitat  oscillatory release technique   catheterization   integrat  neuromusculoskeletal release   catheterization   therapeutic procedure   facilitat  oscillatory release technique   administration procedure   incision and drainage   tonsillectomy   therapeutic procedure   angioplasty   exercise pain management'

# Split the Data

In [373]:
# Import Packages -----

from nltk.corpus import stopwords

my_stop_words = list(set(stopwords.words('english'))) \
                + ['admission', 'date', 'sex'] \
                + ['needed', 'every', 'seen', 'weeks', 'please', 'ml', 'unit', 'small', 'year', 'old', 'cm', 'non', 'mm', 'however']
                # Got the above from my top 100 most predictive words that I wanted to remove

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the data -----

X_train, X_test, y_train, y_test = train_test_split(z[z['CPT_CD'].astype(str).isin(['99221','36556', '99291'])][2].values, z[z['CPT_CD'].astype(str).isin(['99221','36556', '33518'])]['CPT_CD'].astype(str), test_size = .1, random_state = 42)

# Use UMLS Data

In [352]:
z[z['CPT_CD'].astype(str).isin(['99221','36556'])]

,INDEX,CPT_CD,2
0,0,99221,stent placement facilitat oscillatory relea...
1,1,99221,excision facilitat oscillatory release tech...
2,2,99221,facilitat oscillatory release technique cat...
3,3,36556,invasive procedure placement of tube chang...
5,5,99221,therapeutic procedure physical therapy fac...
7,7,36556,neurosurgical procedure reposition proced...
10,10,36556,circumferential supracrestal fiberotomy orga...
12,12,99221,interventional procedure catheterization f...
16,16,99221,facilitat oscillatory release technique tra...
17,17,36556,cardiac lead procedure change procedure ...


# Split the UMLS Data

In [28]:
from nltk.corpus import stopwords

my_stop_words = list(set(stopwords.words('english'))) \
                + ['admission', 'date', 'sex'] \
                + ['needed', 'every', 'seen', 'weeks', 'please', 'ml', 'unit', 'small', 'year', 'old', 'cm', 'non', 'mm', 'however']
                # Got the above from my top 100 most predictive words that I wanted to remove

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = x['value']
y_train = x['variable']
X_test = new_df['TEXT'].values
y_test = new_df['CPT_CD'].astype(str)

# Tokenize the data

In [374]:
# Tokenize the data -----

# Import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)

# Run Naive Bayes

In [375]:
# Use Naive Bayes model -----

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB(alpha=.7)

# Fit and check accuracy
nb_classifier.fit(tfidf_train, y_train)
pred = nb_classifier.predict(tfidf_test)

# Tune NB Model

In [18]:
import numpy as np


def hyperparam_tuning(tfidf_train, y_train, tfidf_test, y_test, nb_classifier):
    for i in np.arange(0,1.1,.1):
        nb_classifier = MultinomialNB()
        nb_classifier.fit(tfidf_train, y_train)
        pred = nb_classifier.predict(tfidf_test)
        print(i)
        print(metrics.accuracy_score(y_test, pred))

hyperparam_tuning(tfidf_train, y_train, tfidf_test, y_test, nb_classifier)  

# Looks like .6-.7 are the best alpha

0.0
0.421076621541547
0.1
0.421076621541547
0.2
0.421076621541547
0.30000000000000004
0.421076621541547
0.4
0.421076621541547
0.5
0.421076621541547
0.6000000000000001
0.421076621541547
0.7000000000000001
0.421076621541547
0.8
0.421076621541547
0.9
0.421076621541547
1.0
0.421076621541547


# Run Logistic Regression

In [38]:
from sklearn.linear_model import LogisticRegression

clf_logist = LogisticRegression(C=.001, random_state = 42, multi_class = 'multinomial', penalty='l2')
clf_logist.fit(tfidf_train, y_train)
logist_pred = clf_logist.predict(tfidf_test)

c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Looking at Feature Names and Weights

In [167]:
# Notes
# sum([np.exp(1)** x for x in nb_classifier.coef_[0]]) # The probability of all the words equals one
# # Taken from here: * https://stackoverflow.com/questions/61586946/how-to-calculate-feature-log-prob-in-the-naive-bayes-multinomialnb

# ------------------------------------------

import numpy as np

def get_feature_rank(tfidf_vectorizer, y_no, nb_classifier):
    
    # Get the feature names
    feature_names = tfidf_vectorizer.get_feature_names()

    # Zip together the first CPT weights with feature names
    feat_with_weights =  sorted(zip(nb_classifier.coef_[y_no], feature_names))
    
    # Print words most responsible for the prediction
#     print('Top 100 \n\n\n\n')
#     top_100_ls = []
    for i in range(100):
        x = feat_with_weights[-i-1]
#         top_100_ls.append(x[1])
#         print(nb_classifier.classes_[y_no], i, round((np.exp(1) ** x[0]),4), x[1])

#     print('\n\n\n\n Bottom 100 \n\n\n\n')
    for i in range(100):
        x = feat_with_weights[i]
#         print(nb_classifier.classes_[y_no], i, round((np.exp(1) ** x[0]),4), x[1])
    
#     min_weight = min([i[0] for i in feat_with_weights])
    
    x = [i[0] for i in feat_with_weights]
    
    median_pred = np.median(x)
          
    return [i[1] for i in feat_with_weights if i[0] <= median_pred] # Minimum weight words
#     return top_100_ls

# Find the least predictive words
def least_pred_words(nb_classifier, tfidf_vectorizer):
    low_wt_stop_ls = []

    for i in range(len(nb_classifier.classes_)):
        low_wt_stop_ls += get_feature_rank(tfidf_vectorizer, i, nb_classifier)

    low_wt_stop_ls = list(set(low_wt_stop_ls))
    return low_wt_stop_ls
    
low_wt_stop_ls = least_pred_words(nb_classifier, tfidf_vectorizer)

# Find top 100 words - doesn't seem to improve the model
def highest_pred_words(nb_classifier, tfidf_vectorizer):
    top_100_ls = []
    for i in range(len(nb_classifier.classes_)):
        top_100_ls += get_feature_rank(tfidf_vectorizer, i, nb_classifier)

    top_100_ls = list(set(top_100_ls))
    return top_100_ls

['citizenship',
 'inutbag',
 'reamin',
 'taspe',
 'traycan',
 'eleveat',
 'interscapular',
 'stenossis',
 'spp',
 'tct',
 'resectable',
 'throacotomy',
 'evisiting',
 'nannulu',
 'tplt',
 'arthrotec',
 'asberger',
 'nnor',
 'sttw',
 'inconssitent',
 'contemplate',
 'whofelt',
 'nreporttemporal',
 'femaile',
 'nbengay',
 'synoviti',
 'flie',
 'controls',
 'navalide',
 'nlipomatosi',
 'endeavour',
 'tractor',
 'expectorated',
 'tegretal',
 'nvalproate',
 'walnut',
 'ncarie',
 'fulminan',
 'nrole',
 'shariro',
 'duckbill',
 'nviridan',
 'supression',
 'mwt',
 'pseudobulbar',
 'propecia',
 'nqpm',
 'kindey',
 'narrowed',
 'maex',
 'inheritable',
 'speculum',
 'intracran',
 'otsc',
 'chornical',
 'nvetriculography',
 'nhycodan',
 'abcces',
 'hypotenion',
 'blank',
 'nfile',
 'herniorrhaphie',
 'nklonipin',
 'ntelapivir',
 'sfebrile',
 'nafld',
 'nkiss',
 'falumouth',
 'hematoemesi',
 'wrote',
 'nspecgr',
 'npostoop',
 'repond',
 'nnwh',
 'reliever',
 'nresultant',
 'nmeasures',
 'digibind',

# Update stop words and tokenize again

In [96]:
my_stop_words += low_wt_stop_ls

tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words, min_df = 3, max_df = .7, sublinear_tf=True)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)


# Create Vocab with top words and tokenize again

In [ ]:
# It reduced test accuracy back to 43% and training went from 50% to 44%

tfidf_vectorizer = TfidfVectorizer(vocabulary=top_100_ls, stop_words=my_stop_words, min_df = 3, max_df = .7, sublinear_tf=True)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)


# Run Naive Bayes again

In [97]:
# Use Naive Bayes model -----

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB(alpha=.7)

# Fit and check accuracy
nb_classifier.fit(tfidf_train, y_train)
pred = nb_classifier.predict(tfidf_test)

# Classification Report

In [368]:
# Create classification report taken from here: https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568
# from sklearn.metrics import classification_report

print('Test')
class_labels = nb_classifier.classes_
print(classification_report(y_test, pred,target_names=class_labels))

print('Training')
pred_x = nb_classifier.predict(tfidf_train)
print(classification_report(y_train, pred_x,target_names=class_labels))

print(y_test)
print(pred)
print(y_train)

Test


ValueError: Number of classes, 2, does not match size of target_names, 3. Try specifying the labels parameter

In [376]:
# metrics.accuracy_score(y_test, pred)
print(y_test, pred)
# """
# V1 NLP Model Accuracy: 0.117
# Wow, I've got a long way to go to improve accuracy
# V2 NLP Model Accuracy: 0.14
# V3 NLP Model Accuracy: .40
# """

# Confusion matrix 
# confusion_mtrx = metrics.confusion_matrix(y_test.astype(str), pred) # 1380, 1380
# confusion_mtrx

37    99221
21    99221
32    99221
Name: CPT_CD, dtype: object ['33518' '36556' '33518']


In [369]:
# Logistical Model accuracy
metrics.accuracy_score(y_test, logist_pred)
# .39



NameError: name 'logist_pred' is not defined

# Vectorize Test

In [84]:
from sklearn.feature_extraction.text import CountVectorizer

vocab = ['love', 'happy', 'run']
count_vectorizer = CountVectorizer(vocabulary = vocab)
x = count_vectorizer.fit_transform(['happy', 'run', 'run', 'run'])
print(x)

  (0, 1)	1
  (1, 2)	1
  (2, 2)	1
  (3, 2)	1


# Splitting out a list in a DataFrame

In [198]:
import pandas as pd

test = pd.DataFrame({'column 1':['The other day, I saw a bear, A great big bear',2,3], 'predictive column':[1,2,3]} )
test['column 1'].str.split(',', expand=True).drop(labels=[1,2], axis=0)
# pd.concat([test['column 1'].str.split(',', expand=True), test['predictive column']], axis=1).melt(id_vars='predictive column').drop('variable', axis=1)

,0,1,2
0,The other day,I saw a bear,A great big bear


,index,SUBJECT_ID,HADM_ID,TEXT,CPT_CD
0,9752,28190,184071.0,admission date discharge date ...,99223
1,8028,23681,198672.0,admission date discharge date ...,99223
2,13259,47569,183424.0,admission date discharge date ...,99223
3,17021,66534,159220.0,admission date discharge date ...,99223
4,8225,24233,159899.0,admission date discharge date ...,99223
...,...,...,...,...,...
125755,23462,99934,176121.0,admission date discharge date ...,99291
125756,23465,99944,185654.0,admission date discharge date ...,99291
125757,23466,99946,157197.0,admission date discharge date ...,99291
125758,23467,99955,108494.0,admission date discharge date ...,99291


# Loop through and Create Txt Files from Df

In [99]:
import numpy as np

new_df = new_df.reset_index()
for index in range(10):
    with open(str(new_df.loc[index, 'CPT_CD']) + '_' + str(index) + '.txt', "w") as text_file:
        text_file.write(new_df.loc[index, 'TEXT'])

# Import all the XMI Files

In [100]:
import glob
import re

procedure_ls = []

proc_regex = re.compile('(?<="T061"\spreferredText=")[\w+\s,]*')

for i in glob.glob('.\\*.xmi'):
    with open(i) as file:
        file_contents = file.read()
        procedures = proc_regex.findall(file_contents)
        procedure_ls.append(list(set(procedures)))

In [101]:
procedure_ls

[['removal technique',
  'Cannulation',
  'Aspiration',
  'Rehabilitation therapy',
  'Anesthesia procedures',
  'General Anesthesia',
  'Injection procedure',
  'Intubation',
  'Interventional procedure',
  'Hand endodontic technique',
  'Dilate procedure',
  'Therapeutic procedure',
  'Placement of stent',
  'Release ',
  'Manipulation procedure'],
 ['Primary operation',
  'Aspiration',
  'Coagulation procedure',
  'Rehabilitation therapy',
  'Interventional procedure',
  'Dilate procedure',
  'Placement of stent',
  'Appendectomy'],
 ['removal technique',
  'Fertilization in Vitro',
  'Percutaneous Transluminal Coronary Angioplasty',
  'Release ',
  'Primary operation',
  'Sodium diet',
  'Embolization, Therapeutic',
  'Interventional procedure',
  'Intubation',
  'Surgical revision',
  'Fluid restriction',
  'Catheterization'],
 ['Vibration ',
  'Primary operation',
  'Reparative closure',
  'Supportive care',
  'Drainage procedure',
  'Craniotomy',
  'Neurosurgical Procedures',
  

In [102]:
new_df.loc[9, 'TEXT']

'admission date                 discharge date      date of birth                sex   m  service surgery  allergie  naprosyn   motrin  attend   chief complaint fever   tachycardia  major surgical or invasive procedure   ct guid  drainage and placement of retroperitoneal catheter    history of present illnes  mr    i  a   m who fell down   step  approximate      week  ago  fall  onto hi  r sid  back  he did not seek medical attention at that time  however  a week and a half later  due to increas  pain  he went to hi  local   where he wa  diagnos  with r sid  rib fracture   given pain medication   and sent home  hi  pain did not subside within a few day  so he present  to hi  pcp    reassur  him and gave him more pain medication  and again discharg  him home   earlier today  he present  to   seeming  septic with fever   tachycardia with hr in the           and leukocytosi  of     he wa  also hyponatremic with a na of      cta perform  at lgh show  bilateral lower lobe infiltrate  that a